In [14]:
import torch
import torch.nn as nn
from math import ceil

In [15]:
base_model = [
    # expand_ratio, channels, reapeats, stride, kernel_size
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]


phi_values = {
    # tuple of : (phi_value, resolution, drop_rate)
    "b0": (0, 224, 0.2), # alpha, beta, gamma, depth = alpha **phi, width = beta **phi, resolution = gamma **phi
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}
    
    

In [16]:
base_model

[[1, 16, 1, 1, 3],
 [6, 24, 2, 2, 3],
 [6, 40, 2, 2, 5],
 [6, 80, 3, 2, 3],
 [6, 112, 3, 1, 5],
 [6, 192, 4, 2, 5],
 [6, 320, 1, 1, 3]]

In [17]:
phi_values

{'b0': (0, 224, 0.2),
 'b1': (0.5, 240, 0.2),
 'b2': (1, 260, 0.3),
 'b3': (2, 300, 0.3),
 'b4': (3, 380, 0.4),
 'b5': (4, 456, 0.4),
 'b6': (5, 528, 0.5),
 'b7': (6, 600, 0.5)}

In [18]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, groups=1
                ):
        super(CNNBlock, self).__init__()
        self.cnn = nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        groups=groups,
        bias=False,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.silu = nn.SiLU() # SiLU <-> Swish

    def forward(self, x):
        return self.silu(self.bn(self.cnn(x)))
        

In [19]:
x = torch.rand(1,3,224,224)
y = CNNBlock(3,9,3,1,1,3)
y

CNNBlock(
  (cnn): Conv2d(3, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3, bias=False)
  (bn): BatchNorm2d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (silu): SiLU()
)

In [20]:
class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # C x H X W -> C X 1 X 1
            nn.Conv2d(in_channels,reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, in_channels, 1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        return x * self.se(x)
        
    

In [21]:
y = SqueezeExcitation(3,10)
y

SqueezeExcitation(
  (se): Sequential(
    (0): AdaptiveAvgPool2d(output_size=1)
    (1): Conv2d(3, 10, kernel_size=(1, 1), stride=(1, 1))
    (2): SiLU()
    (3): Conv2d(10, 3, kernel_size=(1, 1), stride=(1, 1))
    (4): Sigmoid()
  )
)

In [22]:
class InvertedResidualBlock(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride,
        padding,
        expand_ratio,
        reduction = 4,# Squeezeexcitation
        survival_prob = 0.8, # for stochastic depth
    ):
        super(InvertedResidualBlock, self).__init__()
        self.survival_prob = survival_prob
        self.use_residual = in_channels == out_channels and stride == 1
        hidden_dim = in_channels * expand_ratio
        self.expand = in_channels != hidden_dim
        reduced_dim = int(in_channels / reduction)

        if self.expand:
            self.expand_conv = CNNBlock(
                in_channels, hidden_dim, kernel_size=3, stride=1, padding=1,
            )

        self.conv = nn.Sequential(CNNBlock(
                hidden_dim, hidden_dim, kernel_size, stride, padding, groups=hidden_dim,
                                         ),
                SqueezeExcitation(hidden_dim, reduced_dim),
                nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
                nn.BatchNorm2d(out_channels),                 
                )

    def stochastic_depth(self, x):
        if not self.training:
            return x

        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device = x.device) < self.survival_prob

        return torch.div(x, self.survival_prob) * binary_tensor


    def forward(self, inputs):
        x = self.expand_conv(inputs) if self.expand else inputs

        if self.use_residual:
            return self.stochastic_depth(self.conv(x)) + inputs

        else:
            return self.conv(x)
        
        

In [23]:
b = torch.rand(x.shape[0], 1, 1, 1, device = x.device) < 0.8
torch.div(x, 0.8) * b

tensor([[[[0.2081, 0.2716, 1.1669,  ..., 0.8642, 0.4357, 0.8152],
          [0.8216, 0.1873, 0.9800,  ..., 0.0937, 0.0792, 1.0125],
          [0.1267, 0.3654, 1.2110,  ..., 0.9569, 0.8716, 0.3144],
          ...,
          [1.1247, 1.1034, 0.0519,  ..., 1.0487, 0.7342, 0.2173],
          [0.5881, 1.0061, 1.0390,  ..., 0.8671, 0.2833, 0.1685],
          [1.1557, 0.3699, 0.3324,  ..., 0.8595, 0.3806, 0.5475]],

         [[0.8705, 1.1671, 0.3091,  ..., 0.9328, 1.1834, 0.2093],
          [0.4212, 1.0990, 0.5080,  ..., 0.2636, 0.2648, 0.5986],
          [0.2459, 0.2068, 0.0995,  ..., 0.1273, 1.1525, 0.3699],
          ...,
          [1.1806, 0.8299, 0.7076,  ..., 0.0062, 0.9014, 0.4465],
          [0.0516, 0.3459, 0.8891,  ..., 0.9417, 1.0493, 0.1546],
          [0.1275, 1.0843, 1.1652,  ..., 1.0008, 0.0526, 0.3162]],

         [[0.9227, 0.6835, 0.4226,  ..., 0.1419, 0.8452, 0.7273],
          [1.1931, 0.1847, 0.7868,  ..., 0.7125, 1.0765, 1.1192],
          [0.4244, 0.4007, 1.0567,  ..., 0

In [24]:
class EfficientNet(nn.Module):
    def __init__(self, version, num_classes):
        super(EfficientNet, self).__init__()
        width_factor, depth_factor, dropout_rate = self.calculate_factors(version)

        last_channels = ceil(1280 * width_factor)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.features = self.create_features(width_factor, depth_factor, last_channels)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channels, num_classes),
        )
        

    def calculate_factors(self, version, alpha=1.2, beta=1.1):
        phi, res, drop_rate = phi_values[version]
        depth_factor = alpha ** phi
        width_factor = beta ** phi

        return width_factor, depth_factor, drop_rate
    def create_features(self, width_factor, depth_factor, last_channels):
        channels = int(32 * width_factor)
        features = [CNNBlock(3, channels, 3, stride=2, padding=1)]
        in_channels = channels

        for expand_ratio, channels, reapeats, stride, kernel_size in base_model:
            out_channels = 4 * ceil(int(channels*width_factor)/4)
            layers_repeats = ceil(reapeats * depth_factor)

            for layer in range(layers_repeats):
                features.append(
                    InvertedResidualBlock(
                        in_channels,
                        out_channels,
                        expand_ratio = expand_ratio,
                        stride = stride if layer == 0 else 1,
                        kernel_size = kernel_size,
                        padding= kernel_size//2, # if k=1: pad=0, k=3:pad=1, k=5:pad=2
                    )
                )
                
                in_channels = out_channels

        features.append(
            CNNBlock(in_channels, last_channels, kernel_size=1, stride=1, padding=0)
        )
        return nn.Sequential(*features)
                
    def forward(self, x):
        x = self.pool(self.features(x))
        return self.classifier(x.view(x.shape[0], -1))
            
        
        
    
    
        

In [25]:
def test():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    version = "b0"
    phi, res, drop_rate = phi_values[version]
    num_examples, num_classes = 4, 10
    x = torch.randn((num_examples, 3, res, res)).to(device)
    model = EfficientNet(
        version=version,
        num_classes=num_classes,
    ).to(device)

    print(model(x).shape)  # (num_examples, num_classes)

In [26]:
test()

torch.Size([4, 10])
